In [13]:
import pymongo as pym      # Interface with Python <--> MongoDB 
import pandas as pd        # Create Dataframe
import os                  # find files on system
import csv                 # to dump as a csv file       
import json                # convert file to json format
from bson.code import Code  # helps with map-reduce


myclient = pym.MongoClient("mongodb://localhost:27017/")
mydb = myclient["myDB"]
mycol = mydb["Mood_Logger"]

agg_result= mycol.aggregate(
    [
        
    
    {"$group": {
        "_id": {"mood": "$mood", "activity": "$activities"},  
        "count": {"$sum" : 1}  
    }}
        ,
    
    {"$group": {
        "_id": "$_id.mood",  
        "activity_count": {"$push":  { "activity": "$_id.activity", "count": "$count" }}
    }}, 
        {"$unwind": "$activity_count" }
        ,
        {
        
        "$group": {
            "_id": "$_id",
           
            "maxCount":{"$max": "$activity_count.count"},
            "grp": {
            "$push": {
                "mood":"$_id",
                "activity":"$activity_count.activity",
                "count":"$activity_count.count"
            }
        
        }
        
        }
        },
        {
    "$project": {
        "_id": 0,
        "MaxMap": {
            "$setDifference": [{
                    "$map": {
                        "input": "$grp",
                        "as": "grp",
                        "in": {
                            "$cond": [{
                                    "$eq": ["$maxCount", "$$grp.count"]
                                },
                                "$$grp",
                                "false"
                            ]
                        }
                    }
                },
                ["false"]
            ]
        }
 
    }
}, {
    "$unwind": '$MaxMap'
}, {
    "$project": {
        "mood":"$MaxMap.mood",
        "activity":"$MaxMap.activity",
        "count":"$MaxMap.count"
    }
}
    ])

##agg_result to csv
df=pd.DataFrame(columns=['mood', 'activity', 'count'])
for i in agg_result:
    df = df.append(i, ignore_index = True)
    print(i)


print(df)
df.to_csv("/Users/linalouis/Downloads/Result1.csv")

{'mood': 'OVER THE MOON ', 'activity': 'YOUTUBE', 'count': 29}
{'mood': 'OVER THE MOON ', 'activity': 'NEWS UPDATE', 'count': 29}
{'mood': 'WORRIED ', 'activity': 'YOUTUBE', 'count': 20}
{'mood': 'WEAK', 'activity': 'STREAMING', 'count': 27}
{'mood': 'COOL', 'activity': 'YOUTUBE', 'count': 35}
{'mood': 'SAD', 'activity': 'YOUTUBE', 'count': 13}
{'mood': 'SAD', 'activity': 'STREAMING', 'count': 13}
{'mood': 'AWFUL', 'activity': 'FASTING', 'count': 4}
{'mood': 'AWFUL', 'activity': 'AUDIO BOOKS', 'count': 4}
{'mood': 'AWFUL', 'activity': 'STREAMING', 'count': 4}
{'mood': 'AWFUL', 'activity': 'WATCHING SERIES', 'count': 4}
{'mood': 'AWFUL', 'activity': 'REDDIT', 'count': 4}
{'mood': 'AWFUL', 'activity': 'LANGUAGE LEARNING', 'count': 4}
{'mood': 'AWFUL', 'activity': 'MEDITATION', 'count': 4}
{'mood': 'AWFUL', 'activity': 'DOCUMENTARY', 'count': 4}
{'mood': 'AWFUL', 'activity': 'YOUTUBE', 'count': 4}
{'mood': 'CHILL', 'activity': 'YOUTUBE', 'count': 22}
{'mood': 'MEH', 'activity': 'YOUTUBE',